In [7]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [8]:
import requests
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
filename= "spacex_launch_dash.csv"
try:
    response=requests.get(url)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Dataset '{filename}' downloaded successfully.")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the dataset: {e}")

Dataset 'spacex_launch_dash.csv' downloaded successfully.


In [9]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [10]:
# Create a dash application
app = dash.Dash(__name__)

In [11]:
# Dropdown options
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + \
                   [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()]
# Create Dash app
app = dash.Dash(__name__)
# Layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    # TASK 1: Dropdown
    dcc.Dropdown(id='site-dropdown',
                 options=dropdown_options,
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True),
    html.Br(),
    # TASK 2: Pie Chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # TASK 3: Payload RangeSlider
    dcc.RangeSlider(id='payload-slider',
                    min=min_payload,
                    max=max_payload,
                    step=1000,
                    value=[min_payload, max_payload],
                    marks={i: f'{i}' for i in range(int(min_payload), int(max_payload)+1, 2500)},
                    tooltip={"always_visible": True, "placement": "bottom"}),

    # TASK 4: Scatter Plot
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2: Callback for Pie Chart
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        # Group by launch site and sum successful launches
        site_success_counts = filtered_df[filtered_df['class'] == 1]['Launch Site'].value_counts().reset_index()
        site_success_counts.columns = ['Launch Site', 'Successes']

        fig = px.pie(site_success_counts, values='Successes', 
                     names='Launch Site', 
                     title='Total Successful Launches by Site (All Sites)')
        return fig
    else:
        # Filter for specific launch site
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]

        # Count success/failure
        class_counts = site_df['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']
        class_counts['class'] = class_counts['class'].map({1: 'Success', 0: 'Failure'})

        fig = px.pie(class_counts,
                     values='count',
                     names='class',
                     title=f'Total Launch Outcomes for site {entered_site}',
                     color='class',
                     color_discrete_map={'Success': 'lightgreen', 'Failure': 'lightcoral'})
        return fig


In [12]:
# Run the app
if __name__ == '__main__':
    app.run(port=8051)